# Import Data

In [1]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import math

import torch
import torch.nn as nn
import torch.optim as optimr

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from imblearn.over_sampling import SMOTE

In [2]:
zip_path = "ALLFLOWMETER_HIKARI2021.zip"

with zipfile.ZipFile(zip_path, 'r') as zipped_file:
    csv_file_name = zipped_file.namelist()[0]

    with zipped_file.open(csv_file_name) as csv_file:
        df = pd.read_csv(csv_file)

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,uid,originh,originp,responh,responp,flow_duration,fwd_pkts_tot,bwd_pkts_tot,...,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,traffic_category,Label
0,0,0,Cg61Jch3vdz9DBptj,103.255.15.23,13316,128.199.242.104,443,2.207588,15,14,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,29200,65160,0,Bruteforce-XML,1
1,1,1,CdRIlqLWdj35Y9vW9,103.255.15.23,13318,128.199.242.104,443,15.624266,15,14,...,1.534300e+07,1.534300e+07,1.534300e+07,1.534300e+07,0.0,29200,65160,0,Bruteforce-XML,1
2,2,2,CLzp9Khd0Y09Qkgrg,103.255.15.23,13320,128.199.242.104,443,12.203357,14,13,...,1.196814e+07,1.196814e+07,1.196814e+07,1.196814e+07,0.0,29200,65160,0,Bruteforce-XML,1
3,3,3,Cnf1YA4iLB4CSNWB88,103.255.15.23,13322,128.199.242.104,443,9.992448,14,13,...,9.759205e+06,9.759205e+06,9.759205e+06,9.759205e+06,0.0,29200,65160,0,Bruteforce-XML,1
4,4,4,C4ZKvv3fpO72EAOsJ6,103.255.15.23,13324,128.199.242.104,443,7.780611,14,14,...,7.545305e+06,7.545305e+06,7.545305e+06,7.545305e+06,0.0,29200,65160,0,Bruteforce-XML,1


# Data Preprocessing

## Feature Selection

In [4]:
drop_label = ['Unnamed: 0.1', 'Unnamed: 0', 'uid', 'Label']
df = df.drop(labels=drop_label, axis=1)
df.head()

,originh,originp,responh,responp,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,traffic_category
0,103.255.15.23,13316,128.199.242.104,443,2.207588,15,14,6,6,6.794746,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,29200,65160,0,Bruteforce-XML
1,103.255.15.23,13318,128.199.242.104,443,15.624266,15,14,6,6,0.960045,...,158109.181742,1.534300e+07,1.534300e+07,1.534300e+07,1.534300e+07,0.0,29200,65160,0,Bruteforce-XML
2,103.255.15.23,13320,128.199.242.104,443,12.203357,14,13,6,5,1.147225,...,125449.251656,1.196814e+07,1.196814e+07,1.196814e+07,1.196814e+07,0.0,29200,65160,0,Bruteforce-XML
3,103.255.15.23,13322,128.199.242.104,443,9.992448,14,13,6,5,1.401058,...,123182.931318,9.759205e+06,9.759205e+06,9.759205e+06,9.759205e+06,0.0,29200,65160,0,Bruteforce-XML
4,103.255.15.23,13324,128.199.242.104,443,7.780611,14,14,6,5,1.799345,...,126016.885411,7.545305e+06,7.545305e+06,7.545305e+06,7.545305e+06,0.0,29200,65160,0,Bruteforce-XML


In [5]:
df['traffic_category'].unique()

array(['Bruteforce-XML', 'Bruteforce', 'Background', 'Benign', 'Probing',
       'XMRIGCC CryptoMiner'], dtype=object)

In [6]:
numerical_features = df.select_dtypes(include=["float64", "int64"]).columns.tolist()
categorical_features = df.select_dtypes(include=["object", "category"]).columns.tolist()

## Encoding

One-hot untuk target, fitur lain menggunakan label encoding

In [7]:
encoder_label = LabelEncoder()

for col in categorical_features:
    df[col] = encoder_label.fit_transform(df[col])


## Z-Score Standarization

In [8]:
zscore = StandardScaler()

def Standarization(df):
    df_zscore = zscore.fit_transform(df)
    df = pd.DataFrame(df_zscore, columns=df.columns)
    return df

## MinMax Normalization

In [9]:
scaler = MinMaxScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [10]:
df['traffic_category'].nunique()


6

# Model

In [11]:
class Generator(nn.Module):
    def __init__(self, latent_dim, num_classes, data_dim):
        super(Generator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, latent_dim)
        self.model = nn.Sequential(
            nn.Linear(latent_dim * 2, 128),
            nn.ReLU(),
            nn.Linear(128, data_dim),
            nn.Sigmoid()
        )

    def forward(self, noise, labels):
        label_input = self.label_embedding(labels)
        gen_input = torch.cat((noise, label_input), dim=1)
        return self.model(gen_input)


In [12]:
class Discriminator(nn.Module):
    def __init__(self, data_dim, num_classes):
        super(Discriminator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, data_dim)
        self.model = nn.Sequential(
            nn.Linear(data_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, data, labels):
        label_input = self.label_embedding(labels)
        disc_input = torch.cat((data, label_input), dim=1)
        return self.model(disc_input)


In [13]:
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_g_loss = None
        self.best_d_loss = None
        self.counter = 0

    def step(self, g_loss, d_loss):
        if (self.best_g_loss is None or g_loss < self.best_g_loss - self.min_delta) and \
           (self.best_d_loss is None or d_loss < self.best_d_loss - self.min_delta):
            self.best_g_loss = g_loss
            self.best_d_loss = d_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

## Training

In [14]:
# Pisahkan fitur dan label
X = df.drop(columns=['traffic_category'])  # Fitur
y = df['traffic_category']  # Label

# Konversi DataFrame ke numpy array
X = X.to_numpy()
y = y.to_numpy()

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
data = torch.tensor(X, dtype=torch.float32)
labels = torch.tensor(y, dtype=torch.long)
data, labels = data.to(device), labels.to(device)

In [17]:
import optuna
import torch
from torch.utils.data import DataLoader, TensorDataset

data_dim = X.shape[1]
num_classes = len(torch.unique(labels))
def objective(trial):
    # Sampling hyperparameter dari Optuna
    latent_dim = trial.suggest_int("latent_dim", 8, 128, step=8)
    lr_g = trial.suggest_float("lr_g", 1e-5, 1e-2, log=True)
    lr_d = trial.suggest_float("lr_d", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64])
    valid_smoothing = trial.suggest_float("valid_smoothing", 0.7, 0.9, step=0.05) 
    fake_smoothing = trial.suggest_float("fake_smoothing", 0.0, 0.3, step=0.05)

    # Dataset dan DataLoader
    dataset = TensorDataset(data, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Model
    generator = Generator(latent_dim, num_classes, data_dim).to(device)
    discriminator = Discriminator(data_dim, num_classes).to(device)

    # Optimizer
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr_g, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr_d, betas=(0.5, 0.999))
    adversarial_loss = torch.nn.BCELoss().to(device)

    # Training loop (singkat untuk tuning) 
    early_stopping = EarlyStopping(patience=10)
    epochs = 50
    g_losses = []
    d_losses = []

    for epoch in range(epochs):
        for real_data, real_labels in dataloader:
            real_data, real_labels = real_data.to(device), real_labels.to(device)
            batch_size = real_data.size(0)

            # Labels untuk discriminator + smoothing untuk mencegah overconfidence
            valid = torch.ones((batch_size, 1), requires_grad=False).to(device) * valid_smoothing 
            fake = torch.zeros((batch_size, 1), requires_grad=False).to(device) + fake_smoothing

            # ---------------------
            #  Train Discriminator
            # ---------------------
            noise = torch.randn(batch_size, latent_dim).to(device)
            sampled_labels = torch.randint(0, num_classes, (batch_size,)).to(device)

            generated_data = generator(noise, sampled_labels)

            real_loss = adversarial_loss(discriminator(real_data, real_labels), valid)
            fake_loss = adversarial_loss(discriminator(generated_data.detach(), sampled_labels), fake)
            d_loss = (real_loss + fake_loss) / 2

            optimizer_D.zero_grad()
            d_loss.backward()
            optimizer_D.step()

            # -----------------
            #  Train Generator
            # -----------------
            noise = torch.randn(batch_size, latent_dim).to(device)
            sampled_labels = torch.randint(0, num_classes, (batch_size,)).to(device)

            g_loss = adversarial_loss(discriminator(generator(noise, sampled_labels), sampled_labels), valid)

            optimizer_G.zero_grad()
            g_loss.backward()
            optimizer_G.step()

            g_losses.append(g_loss.item())
            d_losses.append(d_loss.item())
        
        # Early stopping check
        if early_stopping.step(g_loss.item(), d_loss.item()):
            print(f"Early stopping at epoch {epoch} dengan D Loss: {d_loss.item()}| G Loss: {g_loss.item()}")
            break

    # Return average generator loss (minimization goal)
    return sum(g_losses) / len(g_losses)

c:\Users\mradi\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5) 

# Menampilkan hasil terbaik
print("Best hyperparameters:", study.best_params)
print("Best loss:", study.best_value)

[I 2024-12-23 16:43:41,552] A new study created in memory with name: no-name-32ffb77c-5a9b-4d01-b142-4109c5897cf7
[I 2024-12-23 16:58:49,892] Trial 0 finished with value: 1.111609896536115 and parameters: {'latent_dim': 40, 'lr_g': 2.0277121963833143e-05, 'lr_d': 0.0013625904398447102, 'batch_size': 64, 'valid_smoothing': 0.75, 'fake_smoothing': 0.25}. Best is trial 0 with value: 1.111609896536115.


Early stopping at epoch 15 dengan D Loss: 0.562335193157196| G Loss: 1.11164128780365


[I 2024-12-23 17:19:27,288] Trial 1 finished with value: 1.1192634034740596 and parameters: {'latent_dim': 120, 'lr_g': 1.1027553064417461e-05, 'lr_d': 0.009693294545595699, 'batch_size': 32, 'valid_smoothing': 0.8999999999999999, 'fake_smoothing': 0.3}. Best is trial 0 with value: 1.111609896536115.


Early stopping at epoch 10 dengan D Loss: 5.305433750152588| G Loss: 1.1192430257797241


[I 2024-12-23 17:32:34,126] Trial 2 finished with value: 0.9499157046192861 and parameters: {'latent_dim': 80, 'lr_g': 3.6716221687272325e-05, 'lr_d': 0.001809283945644003, 'batch_size': 64, 'valid_smoothing': 0.7, 'fake_smoothing': 0.3}. Best is trial 2 with value: 0.9499157046192861.


Early stopping at epoch 13 dengan D Loss: 0.6108644008636475| G Loss: 0.9497835040092468


[I 2024-12-23 17:51:18,958] Trial 3 finished with value: 16.906024840819377 and parameters: {'latent_dim': 88, 'lr_g': 0.0023655400996316664, 'lr_d': 4.156839813303034e-05, 'batch_size': 32, 'valid_smoothing': 0.7999999999999999, 'fake_smoothing': 0.0}. Best is trial 2 with value: 0.9499157046192861.


Early stopping at epoch 10 dengan D Loss: 0.2502027451992035| G Loss: 18.331558227539062


[I 2024-12-23 18:10:48,999] Trial 4 finished with value: 1.973115649010518 and parameters: {'latent_dim': 112, 'lr_g': 8.00138871183803e-05, 'lr_d': 0.0036093000811051705, 'batch_size': 32, 'valid_smoothing': 0.85, 'fake_smoothing': 0.1}. Best is trial 2 with value: 0.9499157046192861.


Early stopping at epoch 10 dengan D Loss: 7.662540435791016| G Loss: 1.9730015993118286
Best hyperparameters: {'latent_dim': 80, 'lr_g': 3.6716221687272325e-05, 'lr_d': 0.001809283945644003, 'batch_size': 64, 'valid_smoothing': 0.7, 'fake_smoothing': 0.3}
Best loss: 0.9499157046192861


In [19]:
optuna.visualization.plot_param_importances(study).show()
optuna.visualization.plot_optimization_history(study).show()

In [20]:
!pip show nbformat


Name: nbformat
Version: 5.10.4
Summary: The Jupyter Notebook format
Home-page: https://jupyter.org
Author: 
Author-email: Jupyter Development Team <jupyter@googlegroups.com>
License: BSD 3-Clause License
        
        - Copyright (c) 2001-2015, IPython Development Team
        - Copyright (c) 2015-, Jupyter Development Team
        
        All rights reserved.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are met:
        
        1. Redistributions of source code must retain the above copyright notice, this
           list of conditions and the following disclaimer.
        
        2. Redistributions in binary form must reproduce the above copyright notice,
           this list of conditions and the following disclaimer in the documentation
           and/or other materials provided with the distribution.
        
        3. Neither the name of the copyright holder nor